# Interactive plotting

Figanos can also be used with the [hvplot](https://hvplot.holoviz.org/) library, which provides a high-level plotting API built on HoloViews to produce interactive plots.

In [ ]:
import pandas as pd
import numpy as np
import xarray as xr
from xclim import ensembles

xr.set_options(keep_attrs=True)

## HvPlot ouranos theme 

In [ ]:
from figanos.hvplot.utils import set_hv_style

set_hv_style("ouranos")

## Timeseries

The main elements of a plot are dependent on four arguments, each accepting dictionaries:

1. `data` : a dictionary containing the Xarray objects and their respective keys, used as labels on the plot.
2. `use_attrs`: a dictionary linking attributes from the Xarray object to plot text elements.
3. `plot_kw` : a dictionary using the same keys as `data` to pass arguments to the underlying plotting function, in this case [hvplot.line](https://hvplot.holoviz.org/reference/tabular/line.html).
4. `opts_kw`: a dictionary using the same keys as `data` plus overlay (to be passed to the combined elements of all `data` values) to pass to the underlying `.opts` [holoviz funciton](https://holoviews.org/user_guide/Customizing_Plots.html).

When labels are passed in `data`, any 'label' argument passed in `plot_kw` will be ignored.

In [ ]:
# creates dataset

time = pd.date_range(start="1960-01-01", end="2020-01-01", periods=10)
np.random.seed(1231)
dat_1 = np.random.rand(10) * 20
dat_2 = np.random.rand(10) * 20
dat_3 = np.random.rand(10) * 20

dt = xr.Dataset(
    data_vars={
        "tas": (
            [
                "realization",
                # 'group',
                "time",
            ],
            # np.array([[dat_1, dat_2], [dat_2, dat_3], [dat_3, dat_1]])
            np.array([dat_1, dat_2, dat_3]),
        )
    },
    coords={
        "time": time,
        "lat": 41,
        "lon": -73,
        # 'group': ['a', 'b'],
        "realization": [0, 1, 2],
    },
)
dt.tas.attrs = {
    "long_name": "Randomly generated time-series",
    "standart_name": "air_temp",
    "description": "Synthetic time-series",
    "units": "degC",
}

data2 = dt + 10

perc = ensembles.ensemble_percentiles(dt, values=[25, 50, 75], split=False)
stat = ensembles.ensemble_mean_std_max_min(dt).drop_vars("tas_stdev")
perc2 = ensembles.ensemble_percentiles(data2, values=[10, 50, 90], split=False)

In [ ]:
from figanos.hvplot import timeseries

timeseries(data2)

In [ ]:
timeseries(data2, legend="in_plot", show_lat_lon="lower left")

In [ ]:
timeseries({"rcp85": data2, "rcp45": dt}, opts_kw={"overlay": {"legend_position": "bottom_right", "legend_cols": 2}}, show_lat_lon="lower left")

In [ ]:
timeseries(
    {"val1": data2, "val2": dt},
    plot_kw={"val1": {"color": "yellow"}, "val2": {"line_width": 5, "color": "teal"}},
    opts_kw={"overlay": {"legend_position": "right"}},
    show_lat_lon="lower left",
)

In [ ]:
timeseries({"ssp245": perc, "ssp585": perc2}, legend="full", show_lat_lon=False, opts_kw={"overlay": {"legend_position": "right"}})